In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

from pymedphys.level1.mudensity import *

DATA_DIRECTORY = "../../tests/data"
DELIVERY_DATA_FILEPATH = os.path.join(DATA_DIRECTORY, 'mu_density_example_arrays.npz')
regress_test_arrays = np.load(DELIVERY_DATA_FILEPATH)

In [ ]:
mu = regress_test_arrays['mu']
mlc = regress_test_arrays['mlc']
jaw = regress_test_arrays['jaw']

leaf_pair_widths = np.array(AGILITY_LEAF_PAIR_WIDTHS)
max_leaf_gap=400
grid_resolution=1
time_steps=50

mu, mlc, jaw = remove_irrelevant_control_points(mu, mlc, jaw)

In [ ]:
full_grid = get_grid(
    max_leaf_gap, grid_resolution, leaf_pair_widths)

In [ ]:
len(full_grid['mlc'])

In [ ]:
len(full_grid['jaw'])

In [ ]:
mu_density = np.zeros((len(full_grid['jaw']), len(full_grid['mlc'])))


for i in range(len(mu) - 1):
    control_point_slice = slice(i, i + 2, 1)
    current_mlc = mlc[control_point_slice, :, :]
    current_jaw = jaw[control_point_slice, :]
    delivered_mu = np.diff(mu[control_point_slice])
    
    (
        grid, grid_leaf_map, current_mlc
    ) = determine_calc_grid_and_adjustments(
        current_mlc, current_jaw, 
        leaf_pair_widths, grid_resolution)
    
    positions = {
        'mlc': {
            1: (-current_mlc[0, :, 0], -current_mlc[1, :, 0]), # left
            -1: (current_mlc[0, :, 1], current_mlc[1, :, 1]) # right
        },
        'jaw': {
            1: (-current_jaw[0::-1, 0], -current_jaw[1::, 0]), # bot
            -1: (current_jaw[0::-1, 1], current_jaw[1::, 1]) # top
        }
    }
    
    blocked_by_device = calc_blocked_by_device(
        grid, positions, grid_resolution, time_steps)
    device_open = calc_device_open(blocked_by_device)
    mlc_open, jaw_open = remap_mlc_and_jaw(device_open, grid_leaf_map)
    open_fraction = calc_open_fraction(mlc_open, jaw_open)
    
    mu_density_of_slice = open_fraction * delivered_mu
    full_grid_mu_density_of_slice = convert_to_full_grid(
        grid, full_grid, mu_density_of_slice)
    
    mu_density += full_grid_mu_density_of_slice

In [ ]:
plt.pcolormesh(full_grid['mlc'], full_grid['jaw'], mu_density)
plt.colorbar()
plt.title('MU density')
plt.xlabel('MLC direction (mm)')
plt.ylabel('Jaw direction (mm)')
plt.axis('equal')
plt.gca().invert_yaxis()

In [ ]:
i = 3
control_point_slice = slice(i, i + 2, 1)

current_mlc = mlc[control_point_slice, :, :]
# current_jaw = jaw[control_point_slice, :]

current_jaw = np.array([
    [0.1, 5],
    [13.2, -4.5]
])

delivered_mu = np.diff(mu[control_point_slice])

(
    grid, grid_leaf_map, current_mlc
) = determine_calc_grid_and_adjustments(
    current_mlc, current_jaw, 
    leaf_pair_widths, grid_resolution)

In [ ]:
current_mlc

In [ ]:
current_jaw

In [ ]:
delivered_mu

In [ ]:
grid

In [ ]:
grid_leaf_map

In [ ]:
positions = {
    'mlc': {
        1: (-current_mlc[0, :, 0], -current_mlc[1, :, 0]), # left
        -1: (current_mlc[0, :, 1], current_mlc[1, :, 1]) # right
    },
    'jaw': {
        1: (-current_jaw[0::-1, 0], -current_jaw[1::, 0]), # bot
        -1: (current_jaw[0::-1, 1], current_jaw[1::, 1]) # top
    }
}

positions

In [ ]:
blocked_by_device = calc_blocked_by_device(grid, positions, grid_resolution, time_steps)

In [ ]:
device_open = calc_device_open(blocked_by_device)

In [ ]:
np.shape(device_open['mlc'])

In [ ]:
np.shape(device_open['jaw'])

In [ ]:
mlc_open, jaw_open = remap_mlc_and_jaw(device_open, grid_leaf_map)

In [ ]:
np.shape(mlc_open)

In [ ]:
np.shape(jaw_open)

In [ ]:
open_fraction = calc_open_fraction(mlc_open, jaw_open)
np.shape(open_fraction)

In [ ]:
mu_density = open_fraction * delivered_mu

In [ ]:
grid['mlc']

In [ ]:
grid['jaw']

In [ ]:
plt.pcolormesh(grid['mlc'], grid['jaw'], mu_density)
plt.colorbar()
plt.title('MU density')
plt.xlabel('MLC direction (mm)')
plt.ylabel('Jaw direction (mm)')
plt.axis('equal')
plt.gca().invert_yaxis()

In [ ]:
full_grid_mu_density = convert_to_full_grid(grid, full_grid, mu_density)

In [ ]:
plt.pcolormesh(full_grid['mlc'], full_grid['jaw'], full_grid_mu_density)
plt.colorbar()
plt.title('MU density')
plt.xlabel('MLC direction (mm)')
plt.ylabel('Jaw direction (mm)')
plt.axis('equal')
plt.gca().invert_yaxis()

In [ ]:
current_mlc

In [ ]:
current_jaw

In [ ]:
AGILITY_LEAF_PAIR_WIDTHS = np.array([
    5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 5, 5, 2.5, 5, 5, 5, 5, 5, 5,
    5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
    5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
    5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2.5, 5, 5, 5, 5, 5, 5
])

In [ ]:
res = 2
leaf_division = AGILITY_LEAF_PAIR_WIDTHS / res
assert np.all(leaf_division.astype(int) == leaf_division)